# GPF (GP Feedback)  v1.0 - Showcase BAD Example

Classification on Age Band-based GP Feedback within a Delta Table using PySpark, SparkSQL, a Lakehouse and it's SQL Endpoint


In [295]:
import mlflow
from pyspark.sql.functions import *
# Set given experiment as the active experiment. If an experiment with this name does not exist, a new experiment with this name is created.
mlflow.set_experiment("gpf-test-2")


StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 297, Finished, Available, Finished)

<Experiment: artifact_location='', creation_time=1727434457880, experiment_id='9345bc6c-7a59-45b1-a8b5-4cd68b5b3dc1', last_update_time=None, lifecycle_stage='active', name='gpf-test-2', tags={}>

In [296]:
df = spark.sql("SELECT * FROM MLLH.dbo.ons_gp_rating_jan24_feb24_dataset_v1 LIMIT 1000")
display(df.limit(5))

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 298, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e37c801d-ca22-4362-a319-151f32e795d6)

In [297]:
dfp = df.select("*").toPandas()
dfp = dfp.drop(['Breakdown_Name'], axis = 1)

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 299, Finished, Available, Finished)

In [298]:
Reponse_set = set(dfp['Response'])
Reponse_set

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 300, Finished, Available, Finished)

{'Do not know',
 'Fairly good',
 'Fairly poor',
 'Neither good nor poor',
 'Prefer not to say',
 'Very good',
 'Very good or fairly good',
 'Very poor',
 'Very poor or fairly poor'}

In [299]:
#GOOD
dfp['Response_Value'] = np.where(dfp['Response'] == 'Very good', 100 , dfp['Response'])
dfp['Response_Value'] = np.where(dfp['Response'] == 'Very good or fairly good', 75 , dfp['Response_Value'])
dfp['Response_Value'] = np.where(dfp['Response'] == 'Fairly good', 50 , dfp['Response_Value'])

#NEUTRAL
dfp['Response_Value'] = np.where(dfp['Response'] == 'Do not know', 0 , dfp['Response_Value'])
dfp['Response_Value'] = np.where(dfp['Response'] == 'Prefer not to say', 0 , dfp['Response_Value'])
dfp['Response_Value'] = np.where(dfp['Response'] == 'Do not know', 0 , dfp['Response_Value'])
dfp['Response_Value'] = np.where(dfp['Response'] == 'Neither good nor poor', 0 , dfp['Response_Value'])

#BAD
dfp['Response_Value'] = np.where(dfp['Response'] == 'Fairly poor', -50 , dfp['Response_Value'])
dfp['Response_Value'] = np.where(dfp['Response'] == 'Very poor or fairly poor', -75 , dfp['Response_Value'])
dfp['Response_Value'] = np.where(dfp['Response'] == 'Very poor', -100 , dfp['Response_Value'])

dfp

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 301, Finished, Available, Finished)

,Response,Age_Band,Happy,Response_Value
0,Do not know,Under 55,0,0
1,Do not know,Under 55,0,0
2,Do not know,Under 55,0,0
3,Do not know,Under 55,0,0
4,Do not know,55 Plus,0,0
...,...,...,...,...
67,Very poor or fairly poor,Under 55,0,-75
68,Very poor or fairly poor,55 Plus,0,-75
69,Very poor or fairly poor,55 Plus,0,-75
70,Very poor or fairly poor,55 Plus,0,-75


In [300]:
dfp['Age_Band'] = np.where(dfp['Age_Band'] == '55 Plus', 1 , dfp['Age_Band'])
dfp['Age_Band'] = np.where(dfp['Age_Band'] == 'Under 55', 0 , dfp['Age_Band'])

dfp

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 302, Finished, Available, Finished)

,Response,Age_Band,Happy,Response_Value
0,Do not know,0,0,0
1,Do not know,0,0,0
2,Do not know,0,0,0
3,Do not know,0,0,0
4,Do not know,1,0,0
...,...,...,...,...
67,Very poor or fairly poor,0,0,-75
68,Very poor or fairly poor,1,0,-75
69,Very poor or fairly poor,1,0,-75
70,Very poor or fairly poor,1,0,-75


In [302]:
X = dfp[['Age_Band','Response_Value']]

y = dfp['Happy']

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 304, Finished, Available, Finished)

In [253]:
X = dfp[['Age_Band_Under 55','Age_Band_55 Plus']]

y = dfp['Response_Value']

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 255, Finished, Available, Finished)

In [339]:
X = np.asarray(X).astype('float32')
print(X[0:5])

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 341, Finished, Available, Finished)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 0.]]


In [309]:
y = np.asarray(y).astype('float32')
print(y)

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 311, Finished, Available, Finished)

[0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [310]:
X.shape

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 312, Finished, Available, Finished)

(72, 2)

In [312]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state = 0)

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 314, Finished, Available, Finished)

In [313]:
import tensorflow as tf
ann = tf.keras.models.Sequential()

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 315, Finished, Available, Finished)

In [314]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) ##input layer

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 316, Finished, Available, Finished)

In [315]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) ##hidden layer 1

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 317, Finished, Available, Finished)

In [316]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) ##output layer

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 318, Finished, Available, Finished)

In [317]:
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.00010)
ann.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 319, Finished, Available, Finished)

In [318]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor = 'accuracy', mode = 'max', verbose = 1, patience = 250)  

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 320, Finished, Available, Finished)

In [385]:
ann.fit(X_train, y_train, batch_size = 64, epochs = 1000, validation_data = (X_test, y_test)
        , callbacks = [early_stop]
       )

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 387, Finished, Available, Finished)

2024-09-27:12:40:09,785 WARNING  [tracking_store.py:153] log_inputs not supported
2024-09-27:12:40:09,786 WARNING  [tracking_store.py:153] log_inputs not supported


Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.4228 - accuracy: 1.0000 - val_loss: 0.4751 - val_accuracy: 1.0000
Epoch 2/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.4228 - accuracy: 1.0000 - val_loss: 0.4751 - val_accuracy: 1.0000
Epoch 3/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.4227 - accuracy: 1.0000 - val_loss: 0.4750 - val_accuracy: 1.0000
Epoch 4/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.4227 - accuracy: 1.0000 - val_loss: 0.4750 - val_accuracy: 1.0000
Epoch 5/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.4227 - accuracy: 1.0000 - val_loss: 0.4749 - val_accuracy: 1.0000
Epoch 6/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.4226 - accuracy: 1.0000 - val_loss: 0.4749 - val_accura

In [386]:
ann.summary()

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 388, Finished, Available, Finished)

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 6)                 18        
                                                                 
 dense_25 (Dense)            (None, 6)                 42        
                                                                 
 dense_26 (Dense)            (None, 1)                 7         
                                                                 
Total params: 67
Trainable params: 67
Non-trainable params: 0
_________________________________________________________________


In [387]:
pd.set_option('display.max_columns', None)
dfp.head(100)

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 389, Finished, Available, Finished)

,Response,Age_Band,Happy,Response_Value
0,Do not know,0,0,0
1,Do not know,0,0,0
2,Do not know,0,0,0
3,Do not know,0,0,0
4,Do not know,1,0,0
...,...,...,...,...
67,Very poor or fairly poor,0,0,-75
68,Very poor or fairly poor,1,0,-75
69,Very poor or fairly poor,1,0,-75
70,Very poor or fairly poor,1,0,-75


In [388]:
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("GPF_Spark") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas
sparkDF=spark.createDataFrame(dfp) 
sparkDF.printSchema()


sparkDF.write.mode("overwrite").format("delta").option("mergeSchema", "true").saveAsTable("ONS_GPF_Dataset_v1")

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 390, Finished, Available, Finished)

root
 |-- Response: string (nullable = true)
 |-- Age_Band: long (nullable = true)
 |-- Happy: string (nullable = true)
 |-- Response_Value: long (nullable = true)



In [404]:
predict_me = [ \
    [ \
        #Over_55?
        1,

        #Response_Value from: -100(BAD) : 0(NEUTRAL) : 100(GOOD)
        0.1
        ]]

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 406, Finished, Available, Finished)

In [405]:
#SUPPRESS SCIENTIFIC NOTATION
np.set_printoptions(precision=16, suppress=True)
#pd.options.display.float_format = '{:.2f}'.format

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 407, Finished, Available, Finished)

In [406]:
#1.0 = You are happy with GPs
#0.5 = You are in the middle
#0.0 = You are not Happy with GPs

ann.predict(predict_me)

StatementMeta(, 3c72c3d9-91f0-4cd2-8918-9ebb618c4ba4, 408, Finished, Available, Finished)

1/1 [==============================] - 0s 26ms/step


array([[0.4181537]], dtype=float32)